In [411]:
import numpy as np

d = 2
p = 3
y = np.identity(d)
X = np.zeros((d,p))
X[:,0] = [1.,0]
X[:,1] = [0,1.]
delta = 1
mag = 1. + delta
angle = np.pi / 4
X[:,2] = [np.sin(angle) * mag,np.cos(angle) * mag]
coeff = get_optimal_x_for_multitask_bp(X, y)

[1. 0. 0. 1.]


In [170]:
X 

array([[1.        , 0.        , 1.41421356],
       [0.        , 1.        , 1.41421356]])

In [124]:
y


array([[1., 0.],
       [0., 1.]])

In [5]:
from sklearn.linear_model import MultiTaskLasso


In [114]:
mtl = MultiTaskLasso(alpha=.0000001, tol = 1e-16, max_iter = 1000000, fit_intercept = False)
mtl.fit(X,y)

/Users/samsonkoelle/miniconda3/envs/montlake_xxx/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:1914: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.127415334491007e-08, tolerance: 2.0000000000000005e-16
  check_random_state(self.random_state), random)


MultiTaskLasso(alpha=1e-07, fit_intercept=False, max_iter=1000000, tol=1e-16)

In [115]:
mtl.coef_

array([[ 0.9525355 , -0.04746429,  0.03356236],
       [-0.04746429,  0.9525355 ,  0.03356236]])

In [116]:
np.sum(np.linalg.norm(mtl.coef_, axis=0))

1.9548989936867542

In [117]:
np.linalg.norm(mtl.predict(X) - y)

2.3232842902115082e-07

In [68]:
#its intuitive that lower lambda means higher value of penalty
#however, by the duality, we expect that the lower lambda means lower value of penalty
#the constraint is tighter on the equality
#we can't find an exact solution... or can we???

In [120]:
# import numpy as np
# from scipy.optimize import linprog

# # Define the problem dimensions
# p, n_tasks = X.shape[1], Y.shape[1]  # Number of features and number of tasks

# # Objective function: minimize the sum of the z variables
# c = np.ones(2 * p * n_tasks)

# # Construct the constraint matrix for the equality Y = X @ beta
# A_eq_list = []
# for j in range(n_tasks):
#     A_eq_list.append(np.hstack([X if i == j else np.zeros_like(X) for i in range(n_tasks)]))
# A_eq = np.vstack(A_eq_list)
# b_eq = Y.flatten()

# # Construct the constraint matrices for the inequality z >= beta and z >= -beta
# A_ineq_list = []
# for j in range(n_tasks):
#     eye_p = np.eye(p)
#     A_ineq_list.append(np.hstack([eye_p if i == j else np.zeros_like(eye_p) for i in range(n_tasks)]))
#     A_ineq_list.append(np.hstack([-eye_p if i == j else np.zeros_like(eye_p) for i in range(n_tasks)]))

# A_ineq = np.vstack(A_ineq_list)
# b_ineq = np.zeros(2 * p * n_tasks)

# # Solve the linear programming problem
# result = linprog(c, A_ub=A_ineq, b_ub=b_ineq, A_eq=A_eq, b_eq=b_eq, method='interior-point')

# # Extract the optimized beta values for each task
# beta_optimized = []
# for j in range(n_tasks):
#     beta_optimized.append(result.x[j * p:(j + 1) * p] - result.x[(j + 1) * p:(j + 2) * p])
# beta_optimized = np.array(beta_optimized)

# # Print the results
# print("True beta:", true_beta)
# print("Optimized beta for each task:", beta_optimized)


In [159]:
# import numpy as np
# from scipy.optimize import linprog

# def get_optimal_x_for_multitask_bp(A, Y):
#     """
#     Solves the multitask basis pursuit problem.
    
#     Parameters:
#     A : numpy.ndarray
#         The dictionary matrix with shape (m, n) where m is the number of observations and n is the number of features.
#     Y : numpy.ndarray
#         The observation matrix with shape (m, t) where t is the number of tasks.
        
#     Returns:
#     X : numpy.ndarray
#         The solution matrix with shape (n, t) where each column represents the solution for the corresponding task.
#     """
#     m, n = A.shape
#     _, t = Y.shape

#     # Create the objective function
#     c = np.concatenate([np.zeros(n * t), np.ones(n * t)])

#     # Create the inequality constraints
#     I = np.eye(n * t)
#     lhs_ineq = np.concatenate([np.concatenate([I, -I], axis=1), np.concatenate([-I, -I], axis=1)], axis=0)
#     rhs_ineq = np.zeros(2 * n * t)

#     # Create the equality constraints
#     lhs_eq = np.concatenate([np.kron(np.eye(t), A), np.zeros((m * t, n * t))], axis=1)
#     rhs_eq = Y.flatten()

#     # Create the bounds for the variables
#     bounds = [(None, None) for _ in range(n * t)] + [(0, None) for _ in range(n * t)]

#     # Solve the linear programming problem
#     print(rhs_eq)
#     res = linprog(c=c, A_ub=lhs_ineq, b_ub=rhs_ineq, A_eq=lhs_eq, b_eq=rhs_eq, bounds=bounds, method="revised simplex", options={'tol': 1e-16})
    
#     if res.success:
#         return res.x[:n * t].reshape((n, t))
#     else:
#         raise ValueError("Linear programming failed to find a solution.")

# # Example usage
# A = np.array([[1, 2], [3, 4], [5, 6]])
# Y = np.array([[1, 2], [3, 4], [5, 6]])

# X = get_optimal_x_for_multitask_bp(A, Y)
# print(X)


In [379]:
import numpy as np
from scipy.optimize import linprog

# Example data (use your actual data here)
# X is the design matrix with shape (n_samples, p)
# Y is the response matrix with shape (n_samples, n_tasks)
n_samples, p = X.shape
n_tasks = y.shape[1]

# Objective function: minimize the sum of the z variables
c = np.ones(2 * p * n_tasks)

# Construct the constraint matrix for the equality Y = X @ beta
A_eq = np.zeros((n_samples * n_tasks, 2 * p * n_tasks))
b_eq = y.flatten()

for j in range(n_tasks):
    A_eq[j * n_samples: (j + 1) * n_samples, j * 2 * p: (j + 1) * 2 * p] = np.hstack([X, -X])

# Construct the constraint matrices for the inequality z >= beta and z >= -beta
A_ineq = np.zeros((2 * p * n_tasks, 2 * p * n_tasks))
b_ineq = np.zeros(2 * p * n_tasks).tolist()

for j in range(n_tasks):
    A_ineq[j * 2 * p: (j + 1) * 2 * p, j * 2 * p: (j + 1) * 2 * p] = np.vstack([
        np.hstack([np.eye(p), -np.eye(p)]),
        np.hstack([-np.eye(p), -np.eye(p)])
    ])

# Solve the linear programming problem
print(A_ineq.shape, A_eq.shape, len(b_ineq), len(b_eq))
result = linprog(c, A_ub=A_ineq, b_ub=b_ineq, A_eq=A_eq, b_eq=b_eq, method='interior-point',options = {'tol': 1e-8})

# Extract the optimized beta values for each task
beta_optimized = np.zeros((p, n_tasks))
for j in range(n_tasks):
    beta_optimized[:, j] = result.x[j * 2 * p: j * 2 * p + p] - result.x[j * 2 * p + p: (j + 1) * 2 * p]

# Print the results
print("Optimized beta for each task:", beta_optimized)


(12, 12) (4, 12) 12 4
Optimized beta for each task: [[ 0.07372908 -0.44352284]
 [-0.44354624  0.07033066]
 [ 0.37344494  0.34024352]]


In [395]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install CVXPY

     |████████████████████████████████| 863 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 11.1 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 190 kB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 142 kB 17.5 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 100 kB 14.9 MB/s ta 0:00:01
  Created wheel for scs: filename=scs-3.2.3-cp36-cp36m-macosx_14_0_x86_64.whl size=112470 sha256=50674c4cd73ed169e4ab9bc95df0b59431f3494c12d6cf62a0d081ca3d9c36b8
  Stored in directory: /Users/samsonkoelle/Library/Caches/pip/wheels/ac/22/ea/a06a82a63ecbc57518b452d3a337ff4034fcb5ce3ca5ae30ac
  Created wheel for ecos: filename=ecos-2.0.14-cp36-cp36m-macosx_10_9_x86_64.whl size=90166 sha256=be11eaefbedb09b

In [401]:
import cvxpy as cp
cp.sum(cp.norm([2,1]).value)

Expression(CONSTANT, NONNEGATIVE, ())

In [409]:
cp.sum(cp.norm([[2,1]], axis = 0)).value

2.23606797749979

In [414]:
import cvxpy as cp
import numpy as np

# Define the optimization variable
beta = cp.Variable((p,d))

# Define the objective function
objective = cp.Minimize(cp.sum(cp.norm(beta, axis = 1)))

# Define the constraints
constraints = [X @ beta == y]

# Form and solve the problem
problem = cp.Problem(objective, constraints)
result = problem.solve()

# The optimized x
beta_optimized = beta.value

print("Optimized x:", beta_optimized)


Optimized x: [[ 0.68898638 -0.31101363]
 [-0.31101362  0.68898637]
 [ 0.21991984  0.21991984]]


In [415]:
# import seaborn as sns
# sns.heatmap(A_eq)

In [416]:
# sns.heatmap(A_ineq)

In [417]:
cp.norm(beta_optimized, axis = 1).value

array([0.75593102, 0.75593101, 0.31101362])

In [418]:
X @ beta_optimized 

array([[ 1.00000000e+00, -2.16967500e-14],
       [-2.57641976e-14,  1.00000000e+00]])

In [421]:
np.linalg.norm(beta_optimized, axis = 1).sum()

1.8228756555718908

In [422]:
help(cp.Problem)

Help on class Problem in module cvxpy.problems.problem:

class Problem(cvxpy.utilities.canonical.Canonical)
 |  A convex optimization problem.
 |  
 |  Problems are immutable, save for modification through the specification
 |  of :class:`~cvxpy.expressions.constants.parameters.Parameter`
 |  
 |  Arguments
 |  ---------
 |  objective : Minimize or Maximize
 |      The problem's objective.
 |  constraints : list
 |      The constraints on the problem variables.
 |  
 |  Method resolution order:
 |      Problem
 |      cvxpy.utilities.canonical.Canonical
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other) -> 'Problem'
 |  
 |  __div__(self, other) -> 'Problem'
 |  
 |  __init__(self, objective:Union[cvxpy.problems.objective.Minimize, cvxpy.problems.objective.Maximize], constraints:Union[List[cvxpy.constraints.constraint.Constraint], NoneType]=None) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __mul__(self, other